<a href="https://colab.research.google.com/github/kabirat/Eglish-Language---Yoruba-Language-Translation/blob/main/Yoruba_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%load_ext autoreload
#%aimport helper, tests
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import collections
import numpy as np
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential

Function Definition for text processing

In [38]:
def test_tokenize(tokenize):
    sentences = [
        'The quick brown fox jumps over the lazy dog .',
        'By Jove , my quick study of lexicography won a prize .',
        'This is a short sentence .']
    tokenized_sentences, tokenizer = tokenize(sentences)
    assert tokenized_sentences == tokenizer.texts_to_sequences(sentences),\
        'Tokenizer returned and doesn\'t generate the same sentences as the tokenized sentences returned. '


def test_pad(pad):
    tokens = [
        [i for i in range(4)],
        [i for i in range(6)],
        [i for i in range(3)]]
    padded_tokens = pad(tokens)
    padding_id = padded_tokens[0][-1]
    true_padded_tokens = np.array([
        [i for i in range(4)] + [padding_id]*2,
        [i for i in range(6)],
        [i for i in range(3)] + [padding_id]*3])
    assert isinstance(padded_tokens, np.ndarray),\
        'Pad returned the wrong type.  Found {} type, expected numpy array type.'
    assert np.all(padded_tokens == true_padded_tokens), 'Pad returned the wrong results.'

    padded_tokens_using_length = pad(tokens, 9)
    assert np.all(padded_tokens_using_length == np.concatenate((true_padded_tokens, np.full((3, 3), padding_id)), axis=1)),\
        'Using length argument return incorrect results'


Encoder - Decoder Function

In [17]:
def _test_model(model, input_shape, output_sequence_length, yoruba_vocab_size):
    #if isinstance(model, Sequential):
        #model = model.model

    assert model.input_shape == (None, *input_shape[1:]),\
        'Wrong input shape. Found input shape {} using parameter input_shape={}'.format(model.input_shape, input_shape)

    assert model.output_shape == (None, output_sequence_length, yoruba_vocab_size),\
        'Wrong output shape. Found output shape {} using parameters output_sequence_length={} and yoruba_vocab_size={}'\
            .format(model.output_shape, output_sequence_length, yoruba_vocab_size)

    #assert len(model.loss_functions) > 0,\
       # 'No loss function set.  Apply the `compile` function to the model.'

    #assert sparse_categorical_crossentropy in model.loss_functions,\
       # 'Not using `sparse_categorical_crossentropy` function for loss.'

def test_simple_model(simple_model):
    input_shape = (137861, 21, 1)
    output_sequence_length = 21
    english_vocab_size = 199
    yoruba_vocab_size = 344

    model = simple_model(input_shape, output_sequence_length, english_vocab_size, yoruba_vocab_size)
    # summarize defined model
    print(model.summary())
    _test_model(model, input_shape, output_sequence_length, yoruba_vocab_size)


def test_embed_model(embed_model):
    input_shape = (137861, 21)
    output_sequence_length = 21
    english_vocab_size = 199
    yoruba_vocab_size = 344

    model = embed_model(input_shape, output_sequence_length, english_vocab_size, yoruba_vocab_size)
    _test_model(model, input_shape, output_sequence_length, yoruba_vocab_size)


def test_encdec_model(encdec_model):
    input_shape = (137861, 15, 1)
    output_sequence_length = 21
    english_vocab_size = 199
    yoruba_vocab_size = 344

    model = encdec_model(input_shape, output_sequence_length, english_vocab_size, yoruba_vocab_size)
    _test_model(model, input_shape, output_sequence_length, yoruba_vocab_size)


def test_bd_model(bd_model):
    input_shape = (137861, 21, 1)
    output_sequence_length = 21
    english_vocab_size = 199
    yoruba_vocab_size = 344

    model = bd_model(input_shape, output_sequence_length, english_vocab_size, yoruba_vocab_size)
    _test_model(model, input_shape, output_sequence_length, yoruba_vocab_size)


def test_model_final(model_final):
    input_shape = (137861, 15)
    output_sequence_length = 21
    english_vocab_size = 199
    yoruba_vocab_size = 344

    model = model_final(input_shape, output_sequence_length, english_vocab_size, yoruba_vocab_size)
    _test_model(model, input_shape, output_sequence_length, yoruba_vocab_size)

Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the yoruba translation.

Preprocess - You'll convert text to sequence of integers.
Models Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
Prediction Run the model on English text.

Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".

If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [18]:
import tensorflow as tf
tf.__version__

#'1.6.0-rc1'

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12790408832399026472
]


Load Data
The data is located in data/small_vocab_en and data/small_vocab_fr. The small_vocab_en file contains English sentences with their yoruba translations in the small_vocab_fr file. Load the English and yoruba data from these files from running the cell below.

In [54]:
# Load English data
with open("drive/My Drive/small_vocab_en.txt",encoding= 'unicode_escape') as f: #Read Yoruba Text
    english_sentences = "".join(f.readlines()).split('\n')
# Load Yoruba data
with open("drive/My Drive/small_vocab_yr.txt",encoding= 'UTF-8') as f: #Read Yoruba Text
    yoruba_sentences = "".join(f.readlines()).split('\n')

print('Dataset Loaded')

Dataset Loaded


Files
Each line in small_vocab_en contains an English sentence with the respective translation in each line of small_vocab_fr. View the first two lines from each file.

In [55]:
for sample_i in range(5):
    print('English sample {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('yoruba sample {}:  {}\n'.format(sample_i + 1, yoruba_sentences[sample_i]))

English sample 1:  Using Ladders â Do You Make These Safety Checks ?
yoruba sample 1:  Lílo Àkàbà — Ǹjẹ́ O Máa Ń Ṣe Àyẹ̀wò Wọ̀nyí Tó Lè Dáàbò Bò Ẹ́ ?

English sample 2:  By Awake !
yoruba sample 2:  Látọwọ́ akọ̀ròyìn Jí !

English sample 3:  correspondent in Ireland
yoruba sample 3:  ní Ireland

English sample 4:  PAUL needed to change a bulb in an outside light fixture of his house .
yoruba sample 4:  PAUL fẹ́ pààrọ̀ gílóòbù iná tó wà lóde ilé ẹ̀ .

English sample 5:  He also needed to clean the outside upstairs windows â his wife had mentioned this several times .
yoruba sample 5:  Ó tún fẹ́ nu àwọn fèrèsé pẹ̀tẹ́ẹ̀sì lọ́wọ́ ita — ó ti pẹ́ tíyàwó ẹ̀ ti ń sọ pé wọ́n ti dọ̀tí .



From looking at the sentences, you can see they have been preprocessed already. The puncuations have been delimited using spaces. All the text have been converted to lowercase. This should save you some time, but the text requires more preprocessing.

Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary. A more complex vocabulary is a more complex problem. Let's look at the complexity of the dataset we'll be working with.

In [56]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
yoruba_words_counter = collections.Counter([word for sentence in yoruba_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} yoruba words.'.format(len([word for sentence in yoruba_sentences for word in sentence.split()])))
print('{} unique yoruba words.'.format(len(yoruba_words_counter)))
print('10 Most common words in the yoruba dataset:')
print('"' + '" "'.join(list(zip(*yoruba_words_counter.most_common(10)))[0]) + '"')

8085776 English words.
69781 unique English words.
10 Most common words in the English dataset:
"." "," "the" "to" "of" "and" "in" "a" "that" "â"

10558055 yoruba words.
51275 unique yoruba words.
10 Most common words in the yoruba dataset:
"," "." "tó" "ń" "àwọn" "pé" "tí" "ṣe" "ní" "máa"


Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:

Tokenize the words into ids
Add padding to make all the sequences the same length.
Time to start preprocessing the data...

Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings. Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number. These are called character and word ids, respectively. Character ids are used for character level models that generate text predictions for each character. A word level model uses word ids that generate text predictions for each word. Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's Tokenizer function. Use this function to tokenize english_sentences and yoruba_sentences in the cell below.

Running the cell will run tokenize on sample data and show output for debugging.

In [57]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length. Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the yoruba sequences have the same length by adding padding to the end of each sequence using Keras's pad_sequences function.

In [58]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')

test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline. Instead, we've provided you with the implementation of the preprocess function.

In [59]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_yoruba_sentences, english_tokenizer, yoruba_tokenizer =\
    preprocess(english_sentences, yoruba_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_yoruba_sequence_length = preproc_yoruba_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
yoruba_vocab_size = len(yoruba_tokenizer.word_index) +1

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max yoruba sentence length:", max_yoruba_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("yoruba vocabulary size:", yoruba_vocab_size)

Data Preprocessed
Max English sentence length: 903
Max yoruba sentence length: 2765
English vocabulary size: 52650
yoruba vocabulary size: 39565


yorubaModels
In this section, you will experiment with various neural network architectures. You will begin by training four relatively simple architectures.

Model 1 is a simple RNN
Model 2 is a RNN with Embedding
Model 3 is a Bidirectional RNN
Model 4 is an optional Encoder-Decoder RNN
After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.

Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want. We want the yoruba translation. The function logits_to_text will bridge the gab between the logits from the neural network to the yoruba translation. You'll be using this function to better understand the output of the neural network.

In [60]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


Model 1: RNN (IMPLEMENTATION)

In [ ]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, yoruba_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param yoruba: Number of unique Yoruba words in the dataset
    :return: Keras model built, but not trained
    """
    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(yoruba_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_yoruba_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_yoruba_sentences.shape[-2], 1))
#tmp_x = tmp_x.reshape((-1, preproc_yoruba_sentences.reshape[1], -1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_yoruba_sequence_length,
    english_vocab_size,
    yoruba_vocab_size)

print(simple_rnn_model.summary())

simple_rnn_model.fit(tmp_x, preproc_yoruba_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], yoruba_tokenizer))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, 21, 256)           198912    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 1024)          263168    
_________________________________________________________________
dropout_2 (Dropout)          (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 21, 344)           352600    
Total params: 814,680
Trainable params: 814,680
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], yoruba_tokenizer))

print("\nCorrect Translation:")
print(yoruba_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Model 2: Embedding (IMPLEMENTATION)
You've turned the words into ids, but there's a better representation of a word. This is called word embeddings. An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [ ]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, yoruba_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param yoruba_vocab_size: Number of unique yoruba words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = tf.keras.Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(yoruba_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

test_embed_model(embed_model)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_yoruba_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_yoruba_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_yoruba_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(yoruba_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_yoruba_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], yoruba_tokenizer))

In [ ]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], yoruba_tokenizer))

print("\nCorrect Translation:")
print(yoruba_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Model 3: Bidirectional RNNs (IMPLEMENTATION)
One restriction of a RNN is that it can't see the future input, only the past. This is where bidirectional recurrent neural networks come in. They are able to see the future data.

In [ ]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, yoruba_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param yoruba_vocab_size: Number of unique yoruba words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.003
    
    # TODO: Build the layers
    model = tf.keras.Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(yoruba_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

test_bd_model(bd_model)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_yoruba_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_yoruba_sentences.shape[-2]))

# TODO: Train and Print prediction(s)
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_yoruba_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(yoruba_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_yoruba_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], yoruba_tokenizer))

In [ ]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], yoruba_tokenizer))

print("\nCorrect Translation:")
print(yoruba_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models. This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence. The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, yoruba_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param yoruba_vocab_size: Number of unique yoruba words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    # Hyperparameters
    learning_rate = 0.001
    
    # Build the layers    
    model = tf.keras.Sequential()
    # Encoder
    model.add(GRU(256, input_shape=input_shape[1:], go_backwards=True))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(GRU(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(yoruba_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)

# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_yoruba_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_yoruba_sentences.shape[-2], 1))

# Train and Print prediction(s)
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_yoruba_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(yoruba_tokenizer.word_index)+1)

encdec_rnn_model.summary()

encdec_rnn_model.fit(tmp_x, preproc_yoruba_sentences, batch_size=1024, epochs=10, validation_split=0.2)


In [ ]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], yoruba_tokenizer))

print("\nCorrect Translation:")
print(yoruba_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [ ]:
def model_final(input_shape, output_sequence_length, english_vocab_size, yoruba_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param yoruba_vocab_size: Number of unique yoruba words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.003
    
    # Build the layers    
    model = tf.keras.Sequential()
    # Embedding
    model.add(Embedding(english_vocab_size, 128, input_length=input_shape[1],
                         input_shape=input_shape[1:]))
    # Encoder
    model.add(Bidirectional(GRU(128)))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(Bidirectional(GRU(128, return_sequences=True)))
    model.add(TimeDistributed(Dense(512, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(yoruba_vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_model_final(model_final)

print('Final Model Loaded')

Prediction (IMPLEMENTATION)

In [ ]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed yoruba data
    :param x_tk: English tokenizer
    :param y_tk: yoruba tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(x.shape,y.shape[1],
                        len(x_tk.word_index)+1,
                        len(y_tk.word_index)+1)
    model.summary()
    model.fit(x, y, batch_size=1024, epochs=25, validation_split=0.2)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_yoruba_sentences, english_tokenizer, yoruba_tokenizer)